<a href="https://colab.research.google.com/github/loicpages/qmII_group_rep/blob/main/final_rainfall_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [517]:
#libraries
import csv
import pandas as pd
import urllib.request    
import requests
from functools import reduce
import plotly.express as px

In [518]:
#create lists for every country which include respective stations. The list contains stations, each of which have monthual data every year.
Madagascar  = ['67009_195109_201712.txt','67012_199901_201712.txt','67019_196104_199412.txt','67023_200901_201712.txt','67025_196010_201708.txt','67027_195109_201712.txt','67037_200901_201311.txt','67067_200801_200912.txt','67072_200801_200905.txt','67073_195109_201708.txt','67083_197501_201712.txt','67085_195109_201612.txt','67095_195109_201712.txt','67107_200901_201310.txt','67113_200901_200912.txt','67117_199901_200806.txt','67131_200801_200912.txt','67137_197111_201712.txt','67143_196201_201712.txt','67152_200102_201708.txt','67161_195109_201708.txt','67197_195109_201612.txt']

Malawi = ['67586_198209_201606.txt','67587_195808_198301.txt','67693_194807_201511.txt','67791_201002_201005.txt']

Mozambique = ['67201_201207_201712.txt', '67205_201208_201712.txt', '67215_194901_201712.txt', '67217_194901_201712.txt','67221_201207_201712.txt',
              '67229_201711_201712.txt','67231_201205_201712.txt','67237_197001_201712.txt','67241_194901_201712.txt','67260_201308_201611.txt',
              '67261_194901_201712.txt','67271_201302_201312.txt','67273_201405_201711.txt','67283_194901_201712.txt','67291_201305_201307.txt','67295_200810_201712.txt','67297_194901_201712.txt',
              '67305_201706_201712.txt','67309_201706_201712.txt','67315_201304_201412.txt','67323_194901_201712.txt','67327_201204_201712.txt',
              '67331_201307_201712.txt','67335_200703_201712.txt','67339_201604_201712.txt','67341_194901_201712.txt', '67346_201306_201712.txt']
 
Namibia = ['68006_201101_201710.txt','68014_196901_201710.txt','68015_201303_201712.txt','68018_201101_201712.txt','68098_201101_201712.txt',
          '68104_198910_200006.txt','68106_201005_201712.txt','68109_201502_201505.txt','68110_195001_201712.txt','68112_196602_201712.txt']
 
Zambia = ['67581_201510_201711.txt','67571_201609_201610.txt','67599_201608_201612.txt','67633_194904_201708.txt','67641_201701_201706.txt',
          '67663_195712_201602.txt','67665_201512_201701.txt','67666_201510_201510.txt']
 
Zimbabwe = ['67761_201104_201110.txt', '67774_194901_199806.txt','67775_197301_201412.txt','67964_194901_201709.txt','67975_198910_201409.txt',
            '67977_201501_201711.txt','67983_194901_201707.txt', '67991_198910_200008.txt']

In [519]:
#Let's create a function that goes through a list, for intance, lists like the ones above, and that adds a https address in front that will permit us to access data.
def append_url_to_lists(country):
  country = list(map(lambda x: 'https://opendata.dwd.de/climate_environment/CDC/observations_global/CLIMAT/monthly/qc/precipitation_total/historical/' + x, country))
  return country

In [520]:
#Let's now call this function for the countries as follows:
Madagascar = append_url_to_lists(Madagascar)
Malawi = append_url_to_lists(Malawi)
Mozambique = append_url_to_lists(Mozambique)
Namibia = append_url_to_lists(Namibia)
Zambia = append_url_to_lists(Zambia)
Zimbabwe = append_url_to_lists(Zimbabwe)

In [521]:
#Let's now create a new function that takes an url in and that creates a dataframe in the format that we want for each station:
def create_df_for_station(url):
  df = pd.read_csv(url, delimiter=';')
  df.drop(df.loc[df['Jahr']<1980].index, inplace=True)
  df['average'] = df.mean(numeric_only=True, axis=1)
  df = df[['Jahr','average']]
  return df

In [522]:
#Let's create another function which will take a country in as a parameter, and average all the stations in that country.
def create_rainfall_df_country(country):

#Creating an empty list that we will append with each station dataframe:
    all_stations_dfs = []
    #for loop iterating through all stations of country:
    for station in country:
      all_stations_dfs.append(create_df_for_station(station))

    #Let's do a few actions to get the data frame in the format that we want
    all_stations_dfs_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Jahr'], how='outer'), all_stations_dfs).fillna(pd.NaT)
    all_stations_dfs_merged.sort_values(by=['Jahr'], inplace = True)
    all_stations_dfs_merged.reset_index(drop = True)
    all_stations_dfs_merged['average'] = all_stations_dfs_merged.mean(axis=1)
    df_new = all_stations_dfs_merged[['average', 'Jahr']]          
    df_new = df_new.rename(columns={'average': 'Rainfall Average (cm/month)', 'Jahr': 'Year'})

    #We make sure to design a return for it to be easier to call the function:
    return df_new

In [523]:
#Let's now call both functions for all our countries:
Madagascar = create_rainfall_df_country(Madagascar)
Malawi = create_rainfall_df_country(Malawi)
Mozambique = create_rainfall_df_country(Mozambique)
Namibia = create_rainfall_df_country(Namibia)
Zambia = create_rainfall_df_country(Zambia)
Zimbabwe = create_rainfall_df_country(Zimbabwe)

In [524]:
#We now import our original dataframe which will be used to plot the rainfall data against our endogenous food availibility data.
data_path = '/content/df_final_use.csv' 
frame = pd.read_csv(data_path)
frame

,Country,Year,Food Availibility per capita (kg/cap/year),Production Quantity (Grains & Ce.+ Root Crops)(1000MT),Import Quantity (Grains & Ce.)(1000MT),Export Quantity (Grains & Ce.)(1000MT),Import Quantity (Root Crops)(1000MT),Export Quantity (Root Crops)(1000MT),Population (Million),Non-Cereal Food Aid (Grain Equiv. 1000MT),Cereal Food Aid (Grain Equiv. 1000MT),Imports (all)(1000MT),Exports (all)(1000MT),Production Quantity per capita (kg/cap/year),Imports (all) per capita (kg/cap/year),Exports (all) per capita (kg/cap/year),Production + net ex-imports (kg/cap/year),Total Food Aid,Total Food Aid per capita (kg/cap/year),Region
0,Afghanistan,1980,NaN,3933.805000,17.100,0.0,0.000000,0.0000,14.186,NaN,NaN,17.100000,0.0000,277.301917,1.205414,0.000000,278.507331,NaN,NaN,Central and East Asia
1,Afghanistan,1981,NaN,3763.130000,96.500,0.0,0.000000,0.0000,13.984,NaN,NaN,96.500000,0.0000,269.102546,6.900744,0.000000,276.003289,NaN,NaN,Central and East Asia
2,Afghanistan,1982,NaN,3602.788000,74.900,0.0,0.000000,0.0000,13.673,NaN,NaN,74.900000,0.0000,263.496526,5.477949,0.000000,268.974475,NaN,NaN,Central and East Asia
3,Afghanistan,1983,NaN,3474.450000,79.503,0.0,0.000000,0.0000,13.300,NaN,NaN,79.503000,0.0000,261.236842,5.977669,0.000000,267.214511,NaN,NaN,Central and East Asia
4,Afghanistan,1984,NaN,3306.778000,21.150,0.0,0.000000,0.0000,12.932,NaN,NaN,21.150000,0.0000,255.705073,1.635478,0.000000,257.340551,NaN,NaN,Central and East Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579,Zimbabwe,2009,141.317483,1677.731196,576.300,0.0,2.163975,0.0144,12.474,69.115038,217.997711,578.463975,0.0144,134.498252,46.373575,0.001154,180.870673,287.112748,23.016895,Southern Africa
2580,Zimbabwe,2010,136.990970,1723.572567,368.600,0.0,2.163975,0.0144,12.571,34.650952,63.648974,370.763975,0.0144,137.107037,29.493594,0.001145,166.599486,98.299925,7.819579,Southern Africa
2581,Zimbabwe,2011,139.933312,1765.470308,640.000,0.0,2.163975,0.0144,12.754,24.445836,32.984747,642.163975,0.0144,138.424832,50.350006,0.001129,188.773709,57.430583,4.502947,Southern Africa
2582,Zimbabwe,2012,142.318797,1255.897663,620.000,0.0,2.163975,0.0144,13.014,16.082141,26.456731,622.163975,0.0144,96.503586,47.807283,0.001107,144.309762,42.538873,3.268701,Southern Africa


In [525]:
#I define a new function that we will use to graph every country:

def graphing_function(country_name, country):

  country_df = frame[frame['Country'] == country_name]
  country_df = country_df[['Year', 'Production Quantity per capita (kg/cap/year)']]
  country_df.set_index('Year', inplace = True)
  final_df = country_df.merge(country, on = 'Year')

  import matplotlib.pyplot as plt
  %matplotlib inline
  x = final_df['Year']
  y1 = final_df['Production Quantity per capita (kg/cap/year)']
  y2 = final_df['Rainfall Average (cm/month)']

  fig, ax1 = plt.subplots()
  ax2 = ax1.twinx()
  curve1 = ax1.plot(x, y1, label = 'Production Quantity per capita (kg/cap/year)', color = 'red')
  curve2 = ax2.plot(x, y2, label = 'Rainfall Average (cm/month)', color = 'blue')
  plt.title('Rainfall and Production in ' + country_name)
  ax1.legend(loc = 'upper left')
  ax1.set_ylabel('Production Quantity per capita (kg/cap/year)')
  ax2.set_ylabel('Rainfall Average (cm/month)')
  ax1.set_xlabel('Year')
  ax2.legend(loc = 'lower right')
  plt.savefig('figure_rainfall_'+ country_name +'.pdf', dpi=300)

In [ ]:
#I call this function on countries in question.
graphing_function('Madagascar', Madagascar)
graphing_function('Malawi', Malawi)
graphing_function('Mozambique', Mozambique)
graphing_function('Namibia', Namibia)
graphing_function('Zambia', Zambia)
graphing_function('Zimbabwe', Zimbabwe)